# Two-Qubit Simulator with Coupler Architecture
*Copyright (c) 2021 Institute for Quantum Computing, Baidu Inc. All Rights Reserved.*


## Outline

This tutorial introduces how to use Quanlse to create the two-qubit simulator with coupler architecture, analyze the $ZZ$ coupling characteristics (a common parasitic coupling in multi-qubit superconducting circuits) and demonstrate the implementation of cross resonance gates and the preparation of Bell states.The outline of this tutorial is as follows:
+ Introduction
+ Preparation
+ Construct Hamiltonian
+ Static $ZZ$ coupling analysis
+ Cross resonance gate analysis
+ Preparation of the Bell State
+ Summary

## Introduction

As the number of qubits in superconducting quantum chips continues to increase, quantum crosstalk between qubits will undermine the fidelity of the gate, leading to an increase in the error rate of the results. Researchers have proposed a tunable coupling architecture to implement a high-fidelity two-qubit gate \[1\]. This structure is composed of two qubits and a frequency-tunable coupler. As the diagram of the structure shown below, $Q_1$ and $Q_2$ are qubits, $C$ is a tunable coupler between them and $g_{1c}, g_{2c}$ are the qubit-coupler coupling strength of coupler with $Q_1$ and $Q_2$, respectively. The qubit-qubit direct coupling strength is $g_{12}$.

![tunable-coupler](figures/tunable-coupler-architectrue.png)

Unlike the classical direct coupling structure, the coupling between qubits can be "turned on" or "turned off" by adjusting the frequency of the coupler in the tuable coupling structure, thus reducing quantum crosstalk noise. So far tunable coupling structures have been widely used in the design of large superconducting quantum computers \[2, 3\].

For the two-qubit with coupler architecture, Quanlse provides a wealth of analysis functions. This tutorial will demonstrate how to analyze the $ZZ$ coupling and the cross resonance gate characteristics, and perpare the bell state using this architecture.

## Preparation

After you have successfully installed Quanlse, you could run the codes below following this tutorial. To run this tutorial, you would need to import the following packages from Quanlse and other commonly-used Python libraries:

In [ ]:
from math import pi
from copy import deepcopy
from numpy import linspace, array
from matplotlib import pyplot as plt

from Quanlse.Simulator.PulseSimQCQ import pulseSimQCQ
from Quanlse.Simulator import PulseModel
from Quanlse.Simulator.PulseSimQCQ import effectiveCoupling, pauliCoefficient
from Quanlse.Utils.Functions import basis, computationalBasisList
from Quanlse.Utils.Plot import plotBarGraph
from Quanlse.QOperation.FixedGate import CNOT
from Quanlse.QOperation.RotationGate import RX

## Construct Hamiltonian

In this section, we will demonstrate how to use Quanlse to construct a physical model of the two-qubit with coupler architecture. The Hamiltonian of this architecture in the lab frame is \[4\]

$$ 
\hat{H}_{\rm sys}= \sum_{i=1,2,{\rm C}}\left(\omega_{i}\hat{a}_{i}^{\dagger}\hat{a}_{i} + \frac{\delta_{i}}{2}\hat{a}_{i}^{\dagger}\hat{a}_{i}^{\dagger}\hat{a}_{i}\hat{a}_{i}\right) + \sum_{i, j=1,2,{\rm C}}^{i \neq j}g_{ij}(\hat{a}_{i} + \hat{a}_{i}^{\dagger})(\hat{a}_{j} + \hat{a}_{j}^{\dagger}),
$$

where the subscripts $1,2,{\rm C}$ represent the two qubits and the coupler, respectively. $\hat{a}_{i}$ and $\hat{a}_{i}^{\dagger}$ are the annihilation and creation operators of the corresponding components (qubits and the coupler); $\omega_{i},\delta_{i}$ represent the frequencies and the magnitudes of the anharmonicity of each component; $g_ {ij}$ represents the coupling strength between different elements.

We use Quanlse to construct the Hamiltonian of the two-qubit with coupler achitecture for the analysis of the $ZZ$ coupling characteristics and implementation of cross resonance gates. We first define the parameters needed by the simulator:

In [ ]:
# Parameters setting
couplerLevel = 3
qubitLevel = 4

subSysNum = 3
subSysLevel = [couplerLevel, qubitLevel, qubitLevel]  # [Coupler, Control Qubit, Target Qubit]

freqDict = {
    0: 6.3 * (2 * pi),  # Coupler frequency (GHz)
    1: 5.1 * (2 * pi),  # Control qubit frequency (GHz)
    2: 4.9 * (2 * pi)  # Target qubit frequency (GHz)
}

anharmDict = {
    0: 0.0 * (2 * pi),  # Coupler anharmonicity (GHz)
    1: -0.33 * (2 * pi),  # Control qubit anharmonicity (GHz)
    2: -0.33 * (2 * pi)  # Target qubit anharmonicity (GHz)
}

couplingMap = {
    (0, 1): 0.098 * (2 * pi),  # Coupling strength of coupler and control qubit (GHz)
    (0, 2): 0.083 * (2 * pi),  # Coupling strength of coupler and target qubit (GHz)
    (1, 2): 0.0025 * (2 * pi)  # Coupling strength of control qubit and target qubit (GHz)
}

Next we use the above parameters to instantiate an object of the `PulseModel` class.

In [ ]:
exampleModel = PulseModel(subSysNum=subSysNum, sysLevel=subSysLevel, qubitFreq=freqDict, qubitAnharm=anharmDict,
                   couplingMap=couplingMap, frameMode='lab', dt=0.01)

After constructing the Hamiltonian, we will demonstrate how to use the analysis tools for the $ZZ$ coupling characteristics and implement cross resonance gates in the two-qubit with coupler architecture.

**Note: To use the analysis tools for two-qubit with coupler architecture, we need to set the number of energy levels of the coupler as 3 and the number of energy levels of the qubit as 4. The index of the coupler should be 0, and the indexes of the qubits should be 1, 2.**

## Static $ZZ$ coupling analysis

To study the properties of the two-qubit with coupler architecture, we need to convert the Hamiltonian of the system to an effective two-qubit direct coupling model. The Hamiltonian is as follows \[2\]，

$$ 
\hat{H}_{\rm eff} = \sum_{i = 1, 2}\left(\tilde{\omega}_{i}\hat{a}_{i}^{\dagger}\hat{a}_{i} + \frac{\delta_{i}}{2}\hat{a}_{i}^{\dagger}\hat{a}_{i}^{\dagger}\hat{a}_{i}\hat{a}_{i}\right) + J(\hat{a}_{1} + \hat{a}_{1}^{\dagger})(\hat{a}_{2} + \hat{a}_{2}^{\dagger}),
$$

where $\tilde{\omega}_{i}$ is the dressed frequency of the qubit, $J$ represents the effective coupling strength between the qubits. The effective coupling is a crucial parameter of a two-qubit structure with a coupler. We can change the effective coupling by tuning the frequency of the coupler to "turn on" or "turn off" the coupling between the two qubits. 

In Quanlse, we can directly obtain the effective coupling through the `effectiveCoupling()` function and input the instantiated `PulseModel` object.

In [ ]:
# Obtain effective coupling strength (GHz)
print(f"Effective coupling strength: {(effectiveCoupling(exampleModel) / 2 / pi)} (GHz)")

The Hamiltonian of the effective coupling model in the two-qubit computational subspace can be decomposed into a sum of Pauli operators (only the ground state and the first excited state of the qubits are considered) \[4\]

$$ 
\hat{H}_{\rm sub} = \alpha_{ZI}\frac{ZI}{2} + \alpha_{IZ}\frac{IZ}{2} + \alpha_{ZZ}\frac{ZZ}{2}, 
$$

where $IZ, ZI, ZZ$ are the Pauli operators in the two-qubit computational subspace. In our notation, the first Pauli operator acts on qubit 1 and the second Pauli operator acts on qubit 2. $\alpha_{ZI}, \alpha_{IZ}, \alpha_{ZZ}$ are the Pauli coefficients.

We can use the `pauliCoefficient` function and input a `PulseModel` object to obtain the Pauli coefficients in the computational subspace.

In [ ]:
print(f"Pauli coefficient: (GHz)")
print(f"  ZI: {pauliCoefficient(exampleModel)['ZI']}")
print(f"  IZ: {pauliCoefficient(exampleModel)['IZ']}")
print(f"  ZZ: {pauliCoefficient(exampleModel)['ZZ']}")

Here, the $ZZ$ term accounts for the crosstalk noise between qubits. In the two-qubit with coupler architecture, we can change the effective coupling strength $J$ between qubits by adjusting the frequency of the coupler to eliminate $ZZ$ interaction. We demonstrate how to choose the appropriate coupler frequency to eliminate the crosstalk noise between qubits. By tuning the coupler frequencies, we can obtain the effective coupling strength $J$ and $ZZ$ interaction strength as a function of coupler frequency.

In [ ]:
freqList = linspace(6.3, 8.8, 200) * (2 * pi)  # The range of different coupler frequencies, in 2 pi GHz

freqDict = deepcopy(freqDict)

effectiveCouplingList = []
zzList = []

# Calculate effective coupling and ZZ interaction for different coupler frequencies
for freq in freqList:
    freqDict[0] = freq
    exampleModel.qubitFreq = freqDict
    effectiveCouplingList.append(effectiveCoupling(exampleModel))
    zzList.append(pauliCoefficient(exampleModel)['ZZ'])
    
plt.figure(figsize=[14, 6])
plt.subplot(121)
plt.plot(freqList / (2 * pi), abs(array(effectiveCouplingList) * 1e3))
plt.xlabel(r'$\omega_c$ (GHz)', fontsize=15)
plt.ylabel(r'$|\ J\ |$ (MHz)', fontsize=15)

plt.title('The absolute values of effective coupling strength', fontsize=15)

plt.subplot(122)
plt.plot(freqList / (2 * pi), abs(array(zzList)) * 1e6, 'r')
plt.xlabel(r'$\omega_c$ (GHz)', fontsize=15)
plt.ylabel(r'$|ZZ|$ (kHz)', fontsize=15)

plt.title('The absolute values of ZZ interaction strength', fontsize=15)
plt.show()

We can see that the effective coupling strength $J$ and the strength of $ZZ$ crosstalk are nearly zero when the coupler frequency is around 8.25 GHz where the crosstalk between these two qubits is therefore eliminated.

## Cross resonance effect analysis

In the two-qubit architecture with a coupler, [iSWAP gate](https://quanlse.baidu.com/#/doc/tutorial-iswap) and [Control-Z gate](https://quanlse.baidu.com/#/doc/tutorial-cz) can be achieved by tuning the coupler frequency or the qubit frequency. The [Cross resonance gate](https://quanlse.baidu.com/#/doc/tutorial-cr) (CR gate) is achieved through a cross resonance effect (CR effect) which is implemented by applying a driving pulse to the control qubit. In this session, we will introduce how to use the analysis tools for the tunable coupling two-qubit architecture in Quanlse. The Hamiltonian of the CR effect in the tunable coupling two-qubit architecture is given by

$$ 
\hat{H}_{\rm CR} = \hat{H}_{\rm sys} + \hat{H}_{\rm drive}, \\
\hat{H}_{\rm drive} = \Omega \cos(\omega_d t)(\hat{a}_{1} + \hat{a}_{1}^{\dagger}), 
$$

where $\hat{H}_{\rm drive}$ is the driving term，$\Omega$ is the driving amplitude and $\omega_d$ is the driving frequency. Here we set qubit 1 to be the control qubit.

The system Hamiltonian can be reduced to the effective CR Hamiltonian by rotating wave approximation (RWA) in the rotating frame and projection onto the computational subspace, expressed as the linear combination of Pauli operators:

$$ 
\hat{H}_{\rm CR,sub} = \alpha_{ZI}\frac{ZI}{2} + \alpha_{IZ}\frac{IZ}{2} + \alpha_{ZZ}\frac{ZZ}{2} + \alpha_{IX}\frac{IX}{2} + \alpha_{IY}\frac{IY}{2} + \alpha_{ZX}\frac{ZX}{2} + \alpha_{ZY}\frac{ZY}{2}.
$$

$\alpha_{ZI}, \alpha_{IZ}, \alpha_{ZZ}, \alpha_{IX}, \alpha_{IY}, \alpha_{ZX}, \alpha_{ZY}$ are the corresponding Pauli coefficients. 
For CR gates, the $ZX$ term is required for the gate implementation, while $ZZ, IX, ZY, IY$ are the noise terms. By adjusting the device parameters or the amplitude of the driving pulse, the intensities of different Pauli operators can be changed, thus eliminating nosie. Here, we will demonstrate how the intensities of the different Pauli operators in the CR effect change depends on the different driving amplitudes `drivingAmp` \[4\].

We can use the `pauliCoefficient()` function and input the instantiated `PulseModel` object and the driving amplitude `drivingAmp` to obtain the Pauli coefficients in the computational subspace. 

In [ ]:
drivingAmpRange =linspace(0, 0.10, 50) * 2 * pi
coeffs = []

for drivingAmp in drivingAmpRange:
    coeffs.append(pauliCoefficient(exampleModel, drivingAmp))
    
ZI = array([coeff['ZI'] for coeff in coeffs])
IZ = array([coeff['IZ'] for coeff in coeffs])
ZZ = array([coeff['ZZ'] for coeff in coeffs])
ZX = array([coeff['ZX'] for coeff in coeffs])
ZY = array([coeff['ZY'] for coeff in coeffs])
IX = array([coeff['IX'] for coeff in coeffs])
IY = array([coeff['IY'] for coeff in coeffs])

In [ ]:
# Define unit
unit = 1e3 / (2 * pi)

plt.figure(figsize=[14, 6])
plt.subplot(121)
[zx, ix] = plt.plot(drivingAmpRange * unit, ZX * 1e3, drivingAmpRange * unit, IX * 1e3)
plt.legend([zx, ix], ['ZX', 'IX'], fontsize=12)
plt.xlabel('Driving amplitude (MHz)', fontsize=15)
plt.ylabel('Interaction strength (MHz)', fontsize=15)
plt.title('Interaction strength vs Driving amplitude', fontsize=15)

plt.subplot(122)
[zz, zy, iz, iy] = plt.plot(drivingAmpRange * unit, ZZ * 1e6, drivingAmpRange * unit, ZY * 1e6, drivingAmpRange * unit, 
                            IZ * 1e6, drivingAmpRange * unit, IY * 1e6)
plt.legend([zz, zy, iz, iy], ['ZZ', 'ZY', 'IZ', 'IY'], fontsize=12)
plt.xlabel('Driving amplitude (MHz)', fontsize=15)
plt.ylabel('Interaction strength (kHz)', fontsize=15)
plt.title('Interaction strength vs Driving amplitude', fontsize=15)

plt.show()

From the above figure, it can be seen that the intensities of the different Pauli operators changes as the amplitude of the driving pulse changes, and by selecting the pulse amplitude corresponding to high $ZX$ intensity and low noise intensities, a high fidelity CR gate can be achieved.

## Preparation of the Bell state

Below we will demonstrate how to use the two-qubit with coupler architecture to prepare a Bell state. A Bell state denotes a two-qubit maximally entangled state\[3\]. An example is the state $ 1 / \sqrt{2} ( | 00 \rangle + | 11 \rangle )$. It can be prepared by the circuit below, in which we use $ X_{\pi / 2} $, namely the $ \pi / 2 $ rotation gate along the $ x $ axis, and a two-qubit CNOT gate.

![Bell-state-CNOT](figures/Bell-state-CNOT.png)

In this section, we use a predefined `PulseModel` instance with the default configuration. For the sake of simplicity, the energy levels of the coupler and the qubits are both set to 2. The position parameter of the coupler is 0, and the position parameter of the qubits are 1, 2 respectively. We first instantiate the `PulseModel` object by calling `pulseSimQCQ`.

In [ ]:
model = pulseSimQCQ(dt = 0.01, frameMode='lab')

For this predefined `PulseModel` instance with the default configuration, Quanlse provides calibrated single-qubit and two-qubit gate operations. We can use these gate operations to define the quantum circuit needed to prepare the Bell state.

In [ ]:
# RX(pi / 2) gate
RX(pi / 2)(model.Q[1])

# CNOT: 1 -> 2
CNOT(model.Q[1], model.Q[2])

The pulse sequence of the quantum circuit defined above is generated by calling the method `model.schedule` . We plot the pulse job by `plot()`. In the figure `uWave1` and `uWave2` are the pulse signals applied to the control qubit and the target qubit, respectively. The coupler channel has no pulse signal, therefore is omitted here. Here, we use the Echo-CR scheme to implement a CR gate \[6\], and construct a CNOT gate by the single-qubit gate and the CR gate (more details see [Cross resonance gate](https://quanlse.baidu.com/#/doc/tutorial-cr)).

In [ ]:
scheJob = model.schedule()
scheJob.plot()

Define the initial state $| \psi \rangle = | 000 \rangle$, in which the three numbers label the state of the coupler, the state of the control qubit and the state of the target qubit respectively. Then, we run the simulation and plot the probability distribution of different outcomes.

In [ ]:
# Run the simulation
psi0 = basis(8, 0)
res = model.simulate(state0=psi0, job=scheJob)

psi1 = res[0]['state']

# Plot the result
popList = [abs(item ** 2) for item in res[0]['state'].T[0]]
basisList = computationalBasisList(3, 2)
plotBarGraph(basisList, popList, "Result", "Outcome", "Population")

As can be seen from the above figure, the measurement outcomes are mainly the $|000\rangle$ state and the $|011\rangle$ state. Therefore, we obtained an approximated Bell state with a fidelity of 80%.

## Summary

We have introduced how to use Quanlse to analyze the $ZZ$ coupling and the cross resonance gate characteristics in the two-qubit with coupler architecture, and demonstrated the preparation of the Bell state based on the this architecture. Users can click this link [tutorial-tunable-coupling-architectrue-two-qubit-simulator.ipynb](https://github.com/baidu/Quanlse/blob/main/Tutorial/EN/tutorial-two-qubit-simulator-with-coupler-architecture.ipynb) to jump to the corresponding GitHub page of this Jupyter Notebook document to get the relevant code. We recommend that users use different parameters from this tutorial to analyze the two-qubit architecture with couplers, simulate the realization of two-qubit gates, and develop analysis tools with a broader range of applications.

## References

\[1\] [Yan Fei, et al. "Tunable coupling scheme for implementing high-fidelity two-qubit gates." *Physical Review Applied* 10.5 (2018): 054062.](https://journals.aps.org/prapplied/abstract/10.1103/PhysRevApplied.10.054062)

\[2\] [Arute, Frank, et al. "Quantum supremacy using a programmable superconducting processor." *Nature* 574.7779 (2019): 505-510.](https://www.nature.com/articles/s41586-019-1666-5)

\[3\] [Wu, Yulin, et al. "Strong quantum computational advantage using a superconducting quantum processor." *arXiv preprint arXiv:2106.14734* (2021).](https://arxiv.org/abs/2106.14734)

\[4\] [Magesan, Easwar, and Jay M. Gambetta. "Effective Hamiltonian models of the cross-resonance gate." *Physical Review A* 101.5 (2020): 052308.](https://link.aps.org/doi/10.1103/PhysRevA.101.052308)

\[5\] ["Bell state" *Wikipedia*.](https://en.wikipedia.org/wiki/Bell_state)

\[6\] [Córcoles, Antonio D., et al. "Process verification of two-qubit quantum gates by randomized benchmarking." *Physical Review A* 87.3 (2013): 030301.](https://journals.aps.org/pra/abstract/10.1103/PhysRevA.87.030301)